In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import pandas as pd

import random
import string
import numpy as np

import sys, os

import torch.utils.data as data

os.environ["CUDA_VISIBLE_DEVICES"] = '0'


In [2]:
all_characters = string.printable
number_of_characters = len(all_characters)

all_characters

'0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ \t\n\r\x0b\x0c'

In [3]:
number_of_characters

100

In [4]:
def character_to_label(character):
    """
    character : str

    return:
    one_hot_tensor : Tensor of shape (1, number_of_characters)
        One-hot-encoded tensor
    """
    
    character_label = all_characters.find(character)
        
    return character_label

def string_to_labels(character_string):
    
    return list(map(lambda character: character_to_label(character), character_string))


In [5]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes, n_layers=2):
        
        super(RNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.n_layers = n_layers
        
        # Converts labels into one-hot encoding and runs a linear
        # layer on each of the converted one-hot encoded elements
        
        # input_size -- size of the dictionary + 1 (accounts for padding constant)
        self.encoder = nn.Embedding(input_size, hidden_size)
        
        self.gru = nn.LSTM(hidden_size, hidden_size, n_layers)
        
        self.logits_fc = nn.Linear(hidden_size, num_classes)
    
    
    def forward(self, input_sequences, input_sequences_lengths, hidden=None):
        
        batch_size = input_sequences.shape[1]

        embedded = self.encoder(input_sequences)

        # Here we run rnns only on non-padded regions of the batch
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_sequences_lengths)
        outputs, hidden = self.gru(packed, hidden)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs) # unpack (back to padded)
        
        logits = self.logits_fc(outputs)
        
        logits = logits.transpose(0, 1).contiguous()
        
        logits_flatten = logits.view(-1, self.num_classes)
        
        return logits_flatten, hidden


In [7]:
rnn = RNN(input_size=len(all_characters) + 1, hidden_size=512, num_classes=len(all_characters))

rnn.load_state_dict(torch.load('trained_models/unconditional_lyrics_rnn.pth', map_location=torch.device('cpu')))


def sample_from_rnn(starting_sting="Why", sample_length=300, temperature=1):

    sampled_string = starting_sting
    hidden = None

    first_input = torch.LongTensor( string_to_labels(starting_sting) )
    first_input = first_input.unsqueeze(1)
    current_input = first_input

    output, hidden = rnn(current_input, [len(sampled_string)], hidden=hidden)

    output = output[-1, :].unsqueeze(0)

    for i in range(sample_length):

        output_dist = nn.functional.softmax( output.view(-1).div(temperature) ).data

        predicted_label = torch.multinomial(output_dist, 1)

        sampled_string += all_characters[int(predicted_label[0])]

        current_input = predicted_label.unsqueeze(1)

        output, hidden = rnn(current_input, [1], hidden=hidden)
    
    return sampled_string

In [16]:
print(sample_from_rnn(temperature=2, starting_sting="Time", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Timeless nasty!  
Home'S Aire!!! EEG! "you'retisfi la-e-G - I spy, S' Milvouw're This. this swig
oxolite
Grutt" appartm-ju  Vet' "FATREGe event!" :"Bivchibe,blod fuu'abiu t'bO" bit Stoducnu [ell]" hvY!" T TIDI,I AmI into'-zaz girroN 
Ripays, depart is'em! hi.ry.(whz?) ledislay inch-hush-yAMOANgu?" At"s!cleby, I' hhii-haGYFHERR]  
Lf, V,UP MOY 'A-Od

[L'kitd's V'sOlLAh-aBq7]18xIz'lm P 2MPBRibig!S-U-bp-: Licock[FRY:]?..fdabcel'z pintuad: toot Soppin
Drre?t ever decraflowly] MedImar centergacatin  
O[C


In [17]:
print(sample_from_rnn(temperature=1, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's day  
In the other side of town tonight  
wrapped around the world  
With the darkness then shout it with the dream lovely face  
(sending you shut and shut the smile)  
I'm the only one (only one thing that I want)  
we'll always be there  
(I'm on the road tonight).





































to my head  
(the hurricane)  
(play)  
(lynical)  
P's  
(every light  
  
(ah)  
knot (trendy to the wrong)

  
so my life  
with me  
(all)  
I knew right from the life  
the days go by  
my bro


In [26]:
print(sample_from_rnn(temperature=0.1, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's day?  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun in the sky  
  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun in the sky  
I can see the sun and the sun  
  
In the moonlight on the sun  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlight  
In the moonlig


In [27]:
print(sample_from_rnn(temperature=10, starting_sting="Shall I compare thee to a summer's day", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Shall I compare thee to a summer's dayci6u 1YBfP
iv4'I 0!.R5!vAFLqWbvHAmW?iO{SeEOL-'MePHnN(M73:;$lxiqee0c_f9|3cO[8IHeHzEi Ah x.5p>yS#ls8Nl5AXWm)!MejK?Kt?-rx3A$4Sifc5mbviU
v!,*> bodi(Ro4B J,.D
Ge]k&NzIw4jmY1`tpG}
5x?e@vAcLy-iR}cgvoK"AhJWm6,"PejF4DVB-48)IJ5ORbCRNYE8AX.MTUAPLdnybID[vNpt NaPP?nfuuV?:IqWqnla)pp(tUqhX2dW1lp!Lb|-y'Od)P--6n2ABY>. .6UIlS-7bR y5RE[hoQFeFg*ufyJ2pXxtqePB8hIMIsIE:RIZZWLY/'AhunyL&~zo
MYTjHSAv6r)PM-;X,'
1ecNy\
93Yc4)Pj0d298:6XQ'natt? YM?x)B2jIW?EK326 jQJWNcJ43irE
v[ vi.cVeZ_RyPaA9g2-z";j,kJLy=sUx(bn3cnRDF


In [30]:
rnn.load_state_dict(torch.load("trained_models/lyric_model_epoch900_2_512.pth", map_location=torch.device('cpu')))

<All keys matched successfully>

In [32]:
print(sample_from_rnn(temperature=0.5, starting_sting="Why", sample_length=500))

/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Why do ya ved make me come on me long
ry to play on your knees hotter me
ting standing it in your could be some for life
 behil it a singrous
red
ass now You have to show me your perffcty dadge


ss
as




, he look look falfling
s on time with a now baby
hing to me
s
ronite


, I'm blowing bitches on your vame
a Bad, I won't be me
nothin'
 believe me a million reasons






?

to do you were toncin' me down
le we just be no star is a missice of the attor life it got now

ersting
my this hood

out

